In [55]:
from dataclasses import dataclass
from typing import List, Dict, Optional, Any, Union
from pydantic import BaseModel, Field
from typing import ClassVar
import os
from pydantic import BaseModel, Field
from typing import ClassVar

cards_by_id = dict()

class Card(BaseModel):
    id: Optional[int] = Field(default=None)
    deck: Optional[str] = Field(default=None)
    question: str
    answer: str
    flag: str
    cardState: str

    def __init__(self, **data):
        super().__init__(**data)

        attempt = 0
        while True:
            attempt += 1
            random_bytes = os.urandom(4)
            random_int = int.from_bytes(random_bytes, byteorder="big")
            if random_int not in cards_by_id:
                cards_by_id[random_int] = self
                self.id = random_int
                break
            if attempt >= 100:
                raise RuntimeError(f"{attempt} attempts of generating a new, unique card id failed.")

    def to_llm_info(self):
        hex_str = f"{value:08x}"  # pad to 8 hex digits
        hex_id = f"{hex_str[:4]}_{hex_str[4:]}"
        s = f"""Card {hex_id} from the deck '{self.deck}'
Question:
{self.question}

Answer:
{self.answer}

Flag: {self.flag}
Card State: {self.cardState}"""
        return s

cards_by_id: dict[int, Card]

class Deck(BaseModel):
    name: str
    cards: List[Card]

class DummyEnvironment(BaseModel):
    decks: List[str]

class CardWithFuzzyMatch(Card):
    __fuzzymatch: Optional[List[str]] = None

class DeckWithCards(BaseModel):
    name: str
    cards: List[Union[CardWithFuzzyMatch, Card]]

class ExpectedResult(BaseModel):
    decks: List[Union[str, DeckWithCards]]

class TestItem(BaseModel):
    name: str
    description: Optional[str] = ""
    environment: str
    queries: List[List[str]]
    params: Optional[Dict[str, Any]] = None
    expected_result: ExpectedResult

class QuestionAnsweringItem(BaseModel):
    name: str
    description: str
    environment: str
    queries: List[List[str]]
    expected_answer: str

class MainModel(BaseModel):
    test_decks: Dict[str, Deck]
    dummy_environments: Dict[str, DummyEnvironment]
    tests: List[TestItem]
    question_answering: Dict[str, List[QuestionAnsweringItem]]


In [53]:
import pathlib

json_path = pathlib.Path("../tests/data/tests.json")
data = MainModel.model_validate_json(json_path.read_text(encoding="utf-8"))

for (deck_key, deck) in data.test_decks.items():
    for card in deck.cards:
        card.deck = deck.name


In [54]:
tmp_idx = 10
expected = data.test_decks["java"].cards[10]
print(cards_by_id[expected.id], expected)
print(3*"\n")

print(expected.to_llm_info())

id=1784539380 deck='Java Programming' question="What is the difference between 'int' and 'Integer'?" answer="'int' is a primitive type; 'Integer' is an object wrapper." flag='None' cardState='New' id=1784539380 deck='Java Programming' question="What is the difference between 'int' and 'Integer'?" answer="'int' is a primitive type; 'Integer' is an object wrapper." flag='None' cardState='New'




Card 075b_cd15 from deck 'Java Programming'
Question:
What is the difference between 'int' and 'Integer'?

Answer:
'int' is a primitive type; 'Integer' is an object wrapper.

Flag: None
Card State: New


## Audio

In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Record audio from microphone
def record_audio(duration=5, fs=16000):
    print("Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    print("Recording finished.")
    return audio.flatten(), fs

# Save audio to a temporary WAV file
def save_wav(audio, fs, filename="temp.wav"):
    wav.write(filename, fs, audio)
    return filename

def create_pipeline():
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model_id = "openai/whisper-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    return pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

pipe = create_pipeline()

# Load and transcribe audio
def transcribe(filename):
    result = pipe(filename)
    print("Transcription:", result)

In [4]:
audio, fs = record_audio(duration=5, fs=16000)
filename = save_wav(audio, fs)
transcribe(filename)

Recording...


/home/fab/Dokumente/Schule/Universität/12. Semester inkl Ilias/Info LLM Praktikum/the-curator/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Recording finished.
Transcription: {'text': ' Hallo, das ist ein Test. Was soll ich eigentlich sagen hier? Zum Schluss zum Warenkorb hinzugefügt. Weißt du was?'}


In [5]:
# import sounddevice as sd
# import numpy as np
#
# def audio_stream(chunk_duration=1, fs=16000):
#     chunk_samples = int(chunk_duration * fs)
#     with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
#         print("Live transcription started. Speak into the microphone.")
#         while True:
#             audio_chunk, _ = stream.read(chunk_samples)
#             yield audio_chunk.flatten()
#
# for chunk in audio_stream():
#     audio_float = (chunk / 32768.0).astype(np.float32)
#     # Pass as dict with 'array' and 'sampling_rate'
#     result = pipe({"array": audio_float, "sampling_rate": 16000})
#     print("You said:", result["text"])

In [18]:
value = 123456789  # example int
hex_str = f"{value:08x}"  # pad to 8 hex digits
formatted = f"{hex_str[:4]}_{hex_str[4:]}"
print(formatted)  # e.g., '075b_cd15'

075b_cd15
